# Experimenting with Cython 

Here are a few example cells showing some of the basics of how to declare variables in Cython. This is not at all comprehensive, but is intended to give you a basis for your own syntax experimentation as you learn the ropes. 

In [1]:
%load_ext Cython

The following code shows how to consistently declare typed memoryviews and numpy arrays. The `trivial_func` does not compute anything useful, it's just acting as a placeholder for you to play around with. Try messing with the values of the `dtype` and/or the double/int/float declarations to see how things work. 

In [9]:
%%cython 

import numpy as np 

def trivial_func(double[:] x):
    cdef double[:] y = np.zeros(5, dtype='f8')
    cdef int[:] z = np.ones(5, dtype='i4')
    return x[0] + y[0] + z[0]

In [10]:
trivial_func(np.ones(4))

2.0

The following code fails to compile because typed memoryviews are not Numpy arrays: you can't just broadcast values willy-nilly.

In [11]:
%%cython 

import numpy as np 

def typed_memoryiews_do_not_support_broadcasting(double[:] x):
    cdef double[:] y = np.zeros(5, dtype='f8')
    return x + y


Error compiling Cython file:
------------------------------------------------------------
...

import numpy as np 

def typed_memoryiews_do_not_support_broadcasting(double[:] x):
    cdef double[:] y = np.zeros(5, dtype='f8')
    return x + y
            ^
------------------------------------------------------------

/Users/aphearin/.ipython/cython/_cython_magic_d08a0720268602291a1b18f7c0b42fd8.pyx:6:13: Invalid operand types for '+' (double[:]; double[:])


The lack of support for features like broadcasting improves performance, but it can be awfully inconvenient. If you want to return x + y, you have to do it the old fashioned way:

In [12]:
%%cython 

import numpy as np 

def summing_x_and_y(double[:] x, double[:] y):
    cdef int i
    cdef int npts = len(y)
    cdef double[:] result = np.zeros(npts, dtype='f8')
    for i in range(npts):
        result[i] = x[i] + y[i]
    return result

In [13]:
summing_x_and_y(np.random.rand(10), np.random.rand(10))

<MemoryView of 'ndarray' at 0x115214ad0>

Notice how the returned value of `summing_x_and_y` is actually a typed memoryview. Since working with Numpy arrays is so much more convenient, it's common to convert your typed memoryviews into Numpy arrays before going back to work in the python layer:

In [16]:
%%cython 

import numpy as np 

def summing_x_and_y_array_result(double[:] x, double[:] y):
    cdef int i
    cdef int npts = len(y)
    cdef double[:] result = np.zeros(npts, dtype='f8')
    for i in range(npts):
        result[i] = x[i] + y[i]
    return np.array(result)

In [17]:
summing_x_and_y_array_result(np.random.rand(10), np.random.rand(10))

array([ 1.08436463,  1.03691581,  0.09408212,  1.32359787,  0.40466598,
        0.4570454 ,  1.33853085,  1.4845437 ,  1.12348052,  1.24053317])